In [ ]:
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from keras.datasets import mnist, fashion_mnist, cifar100, cifar10
from keras.backend import cast_to_floatx

In [ ]:
from keras.models import Model
from keras.layers import Input, Add, Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
#from keras.layers.normalization import BatchNormalization
from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
from keras.regularizers import l2
from keras import backend as K
import random

random.seed(0)
weight_decay = 0.0005

In [ ]:
import abc
import itertools
import numpy as np
from keras.preprocessing.image import apply_affine_transform

## **Load and Preprocessing the data**

In [ ]:
def resize_and_crop_image(input_file, output_side_length, greyscale=False):
    img = cv2.imread(input_file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB if not greyscale else cv2.COLOR_BGR2GRAY)
    height, width = img.shape[:2]
    new_height = output_side_length
    new_width = output_side_length
    if height > width:
        new_height = int(output_side_length * height / width)
    else:
        new_width = int(output_side_length * width / height)
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
    height_offset = (new_height - output_side_length) // 2
    width_offset = (new_width - output_side_length) // 2
    cropped_img = resized_img[height_offset:height_offset + output_side_length,
                              width_offset:width_offset + output_side_length]
    assert cropped_img.shape[:2] == (output_side_length, output_side_length)
    return cropped_img

In [ ]:
def normalize_minus1_1(data):
    return 2*(data/255.) - 1

In [ ]:
def get_channels_axis():
  import keras
  idf = keras.backend.image_data_format()
  if idf == 'channels_first':
      return 1
  assert idf == 'channels_last'
  return 3

In [ ]:
def load_fashion_mnist():
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    X_train = normalize_minus1_1(cast_to_floatx(np.pad(X_train, ((0, 0), (2, 2), (2, 2)), 'constant')))
    X_train = np.expand_dims(X_train, axis=get_channels_axis())
    X_test = normalize_minus1_1(cast_to_floatx(np.pad(X_test, ((0, 0), (2, 2), (2, 2)), 'constant')))
    X_test = np.expand_dims(X_test, axis=get_channels_axis())
    return (X_train, y_train), (X_test, y_test)

## **Saving the results**

In [ ]:
def roc_pr_curve_data(scores, labels):
    scores = scores.flatten()
    labels = labels.flatten()

    scores_pos = scores[labels == 1]
    scores_neg = scores[labels != 1]

    truth = np.concatenate((np.zeros_like(scores_neg), np.ones_like(scores_pos)))
    preds = np.concatenate((scores_neg, scores_pos))
    fpr, tpr, roc_thresholds = roc_curve(truth, preds)
    roc_auc = auc(fpr, tpr)
    print("roc_auc",roc_auc)

    # pr curve where "normal" is the positive class
    precision_norm, recall_norm, pr_thresholds_norm = precision_recall_curve(truth, preds)
    pr_auc_norm = auc(recall_norm, precision_norm)
    print("pr_auc_norm where normal is the positive class",pr_auc_norm)

    # pr curve where "anomaly" is the positive class
    precision_anom, recall_anom, pr_thresholds_anom = precision_recall_curve(truth, -preds, pos_label=0)
    pr_auc_anom = auc(recall_anom, precision_anom)
    print("pr_auc_norm where anomaly is the positive class",pr_auc_anom)

In [ ]:
def get_class_name_from_index(index, dataset_name):
    ind_to_name = {
        'fashion-mnist': ('t-shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag',
                          'ankle-boot'),
    }

    return ind_to_name[dataset_name][index]

## **Transformations**

In [ ]:
class AffineTransformation(object):
    def __init__(self, flip, tx, ty, k_90_rotate):
        self.flip = flip
        self.tx = tx
        self.ty = ty
        self.k_90_rotate = k_90_rotate

    def __call__(self, x):
        res_x = x
        if self.flip:
            res_x = np.fliplr(res_x)
        if self.tx != 0 or self.ty != 0:
            res_x = apply_affine_transform(res_x, tx=self.tx, ty=self.ty,row_axis=0,
    col_axis=1, channel_axis=2, fill_mode='reflect')
        if self.k_90_rotate != 0:
            res_x = np.rot90(res_x, self.k_90_rotate)

        return res_x

In [ ]:
class AbstractTransformer(abc.ABC):
    def __init__(self):
        self._transformation_list = None
        self._create_transformation_list()

    @property
    def n_transforms(self):
        return len(self._transformation_list)

    @abc.abstractmethod
    def _create_transformation_list(self):
        return

    def transform_batch(self, x_batch, t_inds):
        assert len(x_batch) == len(t_inds)

        transformed_batch = x_batch.copy()
        for i, t_ind in enumerate(t_inds):
            transformed_batch[i] = self._transformation_list[t_ind](transformed_batch[i])
        return transformed_batch

In [ ]:
class Transformer(AbstractTransformer):
    def __init__(self, translation_x=8, translation_y=8):
        self.max_tx = translation_x
        self.max_ty = translation_y
        super().__init__()

    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, tx, ty, k_rotate in itertools.product((False, True),
                                                           (0, -self.max_tx, self.max_tx),
                                                           (0, -self.max_ty, self.max_ty),
                                                           range(4)):
            transformation = AffineTransformation(is_flip, tx, ty, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

In [ ]:
class SimpleTransformer(AbstractTransformer):
    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, k_rotate in itertools.product((False, True),
                                                    range(4)):
            transformation = AffineTransformation(is_flip, 0, 0, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

## **The Model: Wide Residual Network**

In [ ]:
def initial_conv(input):
    x = Convolution2D(16, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(input)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    return x


def expand_conv(init, base, k, strides=(1, 1)):
    x = Convolution2D(base * k, (3, 3), padding='same', strides=strides, kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = Convolution2D(base * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    skip = Convolution2D(base * k, (1, 1), padding='same', strides=strides, kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    m = Add()([x, skip])

    return m


def conv1_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv2_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv3_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def create_wide_residual_network(input_dim, nb_classes=100, N=2, k=1, dropout=0.0, verbose=1):
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    ip = Input(shape=input_dim)

    x = initial_conv(ip)
    nb_conv = 4

    x = expand_conv(x, 16, k)
    nb_conv += 2

    for i in range(N - 1):
        x = conv1_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 32, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv2_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 64, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv3_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = AveragePooling2D((8, 8))(x)
    x = Flatten()(x)

    x = Dense(nb_classes, kernel_regularizer=l2(weight_decay), activation='softmax')(x)

    model = Model(ip, x)

    if verbose: print("Wide Residual Network-%d-%d created." % (nb_conv, k))
    return model

## **Experiments**

In [ ]:
import os
import csv
from collections import defaultdict
from glob import glob
from datetime import datetime
from multiprocessing import Manager, freeze_support, Process
import numpy as np
import scipy.stats
from scipy.special import psi, polygamma, gamma
from sklearn.metrics import roc_auc_score
from sklearn.svm import OneClassSVM
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed
from keras.models import Model, Sequential
from tensorflow.keras.layers import Input
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from keras.datasets import mnist, fashion_mnist, cifar100, cifar10
from keras.backend import cast_to_floatx
from tqdm.notebook import tqdm

In [ ]:
def _transformations_experiment(dataset_load_fn, dataset_name, single_class_ind, gpu_q):
    gpu_to_use = gpu_q.get()
    print('training class:',single_class_ind)
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_to_use

    (x_train, y_train), (x_test, y_test) = dataset_load_fn()
    x_train = x_train[:TRAIN_SIZE]
    y_train =  y_train[:TRAIN_SIZE]
    x_test =  x_test[:VAL_SIZE]
    y_test =  y_test[:VAL_SIZE]

    transformer = Transformer(8, 8)
    n, k = (2, 8)
    mdl = create_wide_residual_network(x_train.shape[1:], transformer.n_transforms, n, k)
    mdl.compile('adam',
                'categorical_crossentropy',
                ['acc'])

    x_train_task = x_train[y_train.flatten() == single_class_ind]
    transformations_inds = np.tile(np.arange(transformer.n_transforms), len(x_train_task))
    print('Start of the proposed transformation')
    x_train_task_transformed = transformer.transform_batch(np.repeat(x_train_task, transformer.n_transforms, axis=0),
                                                           transformations_inds)
    print('End of the proposed transformation')
    batch_size = 128
    print('Start of the training')
    mdl.fit(x=x_train_task_transformed, y=to_categorical(transformations_inds),
            batch_size=batch_size, epochs= 5 
            )
    print('End of the training')

    #Estimating the parameters
    def calc_approx_alpha_sum(observations):
        N = observations.shape[0]
        K = observations.shape[1]
        f = np.mean(observations, axis=0)
        log_obs = (np.nan_to_num(np.log(np.clip(observations,10**(-10),None)), copy=True, nan=10, posinf=10**10, neginf=-(10**10)))

        return (N * (K - 1) * (-psi(1))) / (
                N * sum(f * np.log(f)) + sum([np.log(sum(observations[j][:K-1])) for j in range(N)]) - sum([sum(f*log_obs[j]) for j in range(N)]) )
    def calcul_sum_alpha_beta(observations):
      N = observations.shape[0]
      K = observations.shape[1]
      f = np.mean(np.mean(observations, axis=0))
      return (N * (-psi(1))) / (
                2*N*(f * np.log(f)) -f* sum([np.log(sum(observations[j][:K-1])) for j in range(N)]) - f*sum([np.log(observations)[j][K-1] for j in range(N)]) )

    def inv_psi(y, iters=5):
        # initial estimate
        cond = y >= -2.22
        x = cond * (np.exp(y) + 0.5) + (1 - cond) * -1 / (y - psi(1))

        for _ in range(iters):
            x = x - (psi(x) - y) / polygamma(1, x)
        return x

    def fixed_point_beta_louiville_mle(vect_alpha_0, alpha_0, beta_0,y, max_iter=100):
        log_y = np.nan_to_num(np.log(np.clip(y,10**(-10),None)), copy=True, nan=1, posinf=10**10, neginf=-(10**10)).mean(axis=0)
        N = y.shape[0]
        K = y.shape[1]
        alpha_old = alpha_0
        beta_old = beta_0
        vect_alpha_old = vect_alpha_0

        for iterr in range(max_iter):
          #calculate alpha
          aph = psi(alpha_old + beta_old) + (1/N)*sum([np.log(sum(y[j,:K-1])) for j in range(N)]) 
          aph = np.nan_to_num(aph, copy=True, nan=10**3, posinf=10**10, neginf=-(10**10))
          alpha_new = inv_psi(np.clip(aph,None,7), iters=5)
          alpha_new = np.clip(alpha_new,10**(-10),None)

          #calculate beta
          bet = psi(alpha_old + beta_old) + log_y[K-1]
          bet = np.nan_to_num(bet, copy=True, nan=0.1, posinf=10**10, neginf=-(10**10))
          beta_new = inv_psi(np.clip(bet,None,7), iters=5)
          beta_new = np.clip(beta_new,10**(-10),None)

          #calculate the vector alpha
          vect_alpha_new = np.asarray([inv_psi(psi(sum(vect_alpha_old)) + log_y[i] - (1/N)*sum([np.log(sum(y[j,:K-1])) for j in range(N)]) ,iters=5) for i in range(K)])
          vect_alpha_new = np.nan_to_num((np.clip(vect_alpha_new,10**(-10),None)), copy=True, nan=1, posinf=10**10, neginf=-(10**10))
          if np.sqrt(np.sum((vect_alpha_old - vect_alpha_new) ** 2)) < 1e-3 and np.sqrt((alpha_new - alpha_old) ** 2) < 1e-3 and np.sqrt((beta_new - beta_old) ** 2) < 1e-3: 
            break
          #update the parameters
          alpha_old = alpha_new
          beta_old = beta_new
          vect_alpha_old = vect_alpha_new
        return vect_alpha_new, alpha_new, beta_new

    #Normality Score
    def beta_louiville_normality_score(vect_alpha,alpha,beta,y):
      N=y.shape[0]
      K=y.shape[1]
      log_y = np.nan_to_num(np.log(np.clip(y,10**(-10),None)), copy=True, nan=1, posinf=10**10, neginf=-(10**10))
      scores = np.asarray([sum((vect_alpha-1)*log_y[j]) + (beta-1)*log_y[j][K-1] + (alpha-sum(vect_alpha))*np.log(sum(y[j][:k-1]))  for j in range(N)]) #+ np.log(gamma(sum(vect_alpha))) + np.log(gamma(alpha+beta)) - np.log(gamma(alpha)) - np.log(gamma(beta)) - sum([np.log(gamma(vect_alpha[i])) for i in range(K)])
      scores = np.nan_to_num(scores, copy=True, nan=10**7, posinf=10**10, neginf=-(10**10))
      return scores

    scores = np.zeros((len(x_test),))
    observed_data = x_train_task
    for t_ind in range(transformer.n_transforms):
        observed_dirichlet = mdl.predict(transformer.transform_batch(observed_data, [t_ind] * len(observed_data)),
                                         batch_size=128)
        alpha_sum_approx = np.abs(calc_approx_alpha_sum(observed_dirichlet))
        alpha_sum_approx = np.nan_to_num(alpha_sum_approx, copy=True, nan=1, posinf=10**10, neginf=-10*10)
        vect_alpha_0 = np.clip(observed_dirichlet.mean(axis=0) * alpha_sum_approx,10**(-7),10**7)
        alpha_0 = 0.01
        beta_0 = 0.01

        mle_vect_alpha_t,mle_alpha_t,mle_beta_t = fixed_point_beta_louiville_mle(vect_alpha_0, alpha_0, beta_0,observed_dirichlet,max_iter=10)
        mle_vect_alpha_t=np.nan_to_num(mle_vect_alpha_t, copy=True, nan=1, posinf=10**10, neginf=-10*10)
        mle_alpha_t= np.nan_to_num(mle_alpha_t, copy=True, nan=1, posinf=10**10, neginf=-10*10)
        mle_beta_t= np.nan_to_num(mle_beta_t, copy=True, nan=1, posinf=10**10, neginf=-10**10)

        x_test_p = mdl.predict(transformer.transform_batch(x_test, [t_ind] * len(x_test)),
                               batch_size=128)
        scores += beta_louiville_normality_score(mle_vect_alpha_t,mle_alpha_t,mle_beta_t, x_test_p)
        scores = np.nan_to_num(scores, copy=True, nan=10**7, posinf=10**10, neginf=-(10**10))

    scores /= transformer.n_transforms
    labels = y_test.flatten() == single_class_ind
    roc_pr_curve_data(scores, labels)

    gpu_q.put(gpu_to_use)

In [ ]:
def run_experiments(load_dataset_fn, dataset_name, q, n_classes):
    print("START _transformations_experiment ")
    n_runs = 3
    # Transformations
    for i in range(n_runs):
        print('Run number:',i+1)
        processes = [Process(target=_transformations_experiment,
                             args=(load_dataset_fn, dataset_name, c, q)) for c in range(n_classes)]
        for p in processes:
            p.start()
        for p in processes:
            p.join()
    print("END _transformations_experiment ")

In [ ]:
RESULTS_DIR = ''
TRAIN_SIZE = 10000
VAL_SIZE = 1000
BATCH_SIZE = 128
N_EPOCHS = 5

In [ ]:
if __name__ == '__main__':
    freeze_support()
    N_GPUS = 1
    man = Manager()
    q = man.Queue(N_GPUS)
    for g in range(N_GPUS):
        q.put(str(g))

    experiments_list = [ 
        (load_fashion_mnist, 'fashion-mnist', 10),
    ]

    for data_load_fn, dataset_name, n_classes in experiments_list:
        run_experiments(data_load_fn, dataset_name, q, n_classes)

START _transformations_experiment 
Run number: 1
training class: 0
Wide Residual Network-16-8 created.
Start of the proposed transformation
End of the proposed transformation
Start of the training
Epoch 1/5


204/530 [==========>...................] - ETA: 29:05 - loss: 8.8392 - acc: 0.01 - ETA: 20:50 - loss: 8.7428 - acc: 0.01 - ETA: 19:48 - loss: 8.6467 - acc: 0.01 - ETA: 19:16 - loss: 8.5521 - acc: 0.02 - ETA: 18:44 - loss: 8.4588 - acc: 0.02 - ETA: 18:20 - loss: 8.3662 - acc: 0.03 - ETA: 18:04 - loss: 8.2758 - acc: 0.04 - ETA: 17:55 - loss: 8.1877 - acc: 0.04 - ETA: 17:47 - loss: 8.1013 - acc: 0.05 - ETA: 17:40 - loss: 8.0172 - acc: 0.06 - ETA: 17:34 - loss: 7.9352 - acc: 0.07 - ETA: 17:29 - loss: 7.8556 - acc: 0.07 - ETA: 17:23 - loss: 7.7779 - acc: 0.07 - ETA: 17:18 - loss: 7.7023 - acc: 0.07 - ETA: 17:12 - loss: 7.6291 - acc: 0.08 - ETA: 17:10 - loss: 7.5572 - acc: 0.08 - ETA: 17:06 - loss: 7.4873 - acc: 0.08 - ETA: 17:01 - loss: 7.4189 - acc: 0.09 - ETA: 16:57 - loss: 7.3527 - acc: 0.09 - ETA: 16:58 - loss: 7.2883 - acc: 0.09 - ETA: 16:54 - loss: 7.2251 - acc: 0.09 - ETA: 16:55 - loss: 7.1638 - acc: 0.10 - ETA: 16:51 - loss: 7.1045 - acc: 0.10 - ETA: 16:48 - loss: 7.0466 - acc: 0.10

408/530 [======================>.......] - ETA: 10:27 - loss: 3.0097 - acc: 0.36 - ETA: 10:25 - loss: 3.0020 - acc: 0.36 - ETA: 10:23 - loss: 2.9942 - acc: 0.36 - ETA: 10:21 - loss: 2.9865 - acc: 0.36 - ETA: 10:19 - loss: 2.9788 - acc: 0.36 - ETA: 10:17 - loss: 2.9711 - acc: 0.36 - ETA: 10:15 - loss: 2.9635 - acc: 0.36 - ETA: 10:13 - loss: 2.9561 - acc: 0.36 - ETA: 10:11 - loss: 2.9486 - acc: 0.36 - ETA: 10:09 - loss: 2.9410 - acc: 0.37 - ETA: 10:07 - loss: 2.9338 - acc: 0.37 - ETA: 10:05 - loss: 2.9265 - acc: 0.37 - ETA: 10:04 - loss: 2.9193 - acc: 0.37 - ETA: 10:02 - loss: 2.9125 - acc: 0.37 - ETA: 10:00 - loss: 2.9059 - acc: 0.37 - ETA: 9:58 - loss: 2.8989 - acc: 0.3735 - ETA: 9:56 - loss: 2.8921 - acc: 0.374 - ETA: 9:54 - loss: 2.8851 - acc: 0.374 - ETA: 9:52 - loss: 2.8781 - acc: 0.375 - ETA: 9:50 - loss: 2.8710 - acc: 0.375 - ETA: 9:48 - loss: 2.8640 - acc: 0.376 - ETA: 9:46 - loss: 2.8578 - acc: 0.376 - ETA: 9:44 - loss: 2.8510 - acc: 0.377 - ETA: 9:42 - loss: 2.8444 - acc: 0.37

530/530 [==============================] - ETA: 3:53 - loss: 2.1162 - acc: 0.472 - ETA: 3:51 - loss: 2.1135 - acc: 0.473 - ETA: 3:49 - loss: 2.1110 - acc: 0.473 - ETA: 3:47 - loss: 2.1084 - acc: 0.473 - ETA: 3:45 - loss: 2.1059 - acc: 0.474 - ETA: 3:43 - loss: 2.1032 - acc: 0.475 - ETA: 3:41 - loss: 2.1007 - acc: 0.475 - ETA: 3:39 - loss: 2.0983 - acc: 0.476 - ETA: 3:37 - loss: 2.0958 - acc: 0.476 - ETA: 3:35 - loss: 2.0931 - acc: 0.477 - ETA: 3:33 - loss: 2.0906 - acc: 0.477 - ETA: 3:31 - loss: 2.0880 - acc: 0.478 - ETA: 3:30 - loss: 2.0856 - acc: 0.478 - ETA: 3:28 - loss: 2.0831 - acc: 0.479 - ETA: 3:26 - loss: 2.0806 - acc: 0.479 - ETA: 3:24 - loss: 2.0782 - acc: 0.480 - ETA: 3:22 - loss: 2.0757 - acc: 0.481 - ETA: 3:20 - loss: 2.0730 - acc: 0.481 - ETA: 3:18 - loss: 2.0703 - acc: 0.482 - ETA: 3:16 - loss: 2.0679 - acc: 0.482 - ETA: 3:14 - loss: 2.0653 - acc: 0.483 - ETA: 3:12 - loss: 2.0628 - acc: 0.484 - ETA: 3:10 - loss: 2.0603 - acc: 0.484 - ETA: 3:08 - loss: 2.0580 - acc: 0.485

204/530 [==========>...................] - ETA: 16:50 - loss: 1.0241 - acc: 0.68 - ETA: 16:53 - loss: 0.9729 - acc: 0.73 - ETA: 16:50 - loss: 0.9850 - acc: 0.72 - ETA: 16:47 - loss: 0.9900 - acc: 0.71 - ETA: 16:47 - loss: 0.9764 - acc: 0.72 - ETA: 16:45 - loss: 0.9734 - acc: 0.72 - ETA: 16:43 - loss: 0.9652 - acc: 0.72 - ETA: 16:39 - loss: 0.9628 - acc: 0.72 - ETA: 16:39 - loss: 0.9627 - acc: 0.72 - ETA: 16:37 - loss: 0.9631 - acc: 0.72 - ETA: 16:36 - loss: 0.9633 - acc: 0.72 - ETA: 16:34 - loss: 0.9619 - acc: 0.72 - ETA: 16:33 - loss: 0.9577 - acc: 0.72 - ETA: 16:30 - loss: 0.9567 - acc: 0.72 - ETA: 16:34 - loss: 0.9530 - acc: 0.73 - ETA: 16:33 - loss: 0.9490 - acc: 0.73 - ETA: 16:30 - loss: 0.9448 - acc: 0.73 - ETA: 16:27 - loss: 0.9477 - acc: 0.73 - ETA: 16:25 - loss: 0.9498 - acc: 0.73 - ETA: 16:22 - loss: 0.9496 - acc: 0.73 - ETA: 16:20 - loss: 0.9488 - acc: 0.73 - ETA: 16:18 - loss: 0.9483 - acc: 0.73 - ETA: 16:15 - loss: 0.9486 - acc: 0.73 - ETA: 16:13 - loss: 0.9493 - acc: 0.73

...

408/563 [====================>.........] - ETA: 11:32 - loss: 0.1627 - acc: 0.99 - ETA: 11:30 - loss: 0.1627 - acc: 0.99 - ETA: 11:28 - loss: 0.1628 - acc: 0.99 - ETA: 11:26 - loss: 0.1627 - acc: 0.99 - ETA: 11:24 - loss: 0.1631 - acc: 0.99 - ETA: 11:22 - loss: 0.1630 - acc: 0.99 - ETA: 11:20 - loss: 0.1631 - acc: 0.99 - ETA: 11:18 - loss: 0.1632 - acc: 0.99 - ETA: 11:16 - loss: 0.1633 - acc: 0.99 - ETA: 11:14 - loss: 0.1633 - acc: 0.99 - ETA: 11:12 - loss: 0.1633 - acc: 0.99 - ETA: 11:11 - loss: 0.1633 - acc: 0.99 - ETA: 11:09 - loss: 0.1634 - acc: 0.99 - ETA: 11:07 - loss: 0.1633 - acc: 0.99 - ETA: 11:05 - loss: 0.1633 - acc: 0.99 - ETA: 11:03 - loss: 0.1634 - acc: 0.99 - ETA: 11:01 - loss: 0.1634 - acc: 0.99 - ETA: 10:59 - loss: 0.1635 - acc: 0.99 - ETA: 10:57 - loss: 0.1635 - acc: 0.99 - ETA: 10:55 - loss: 0.1634 - acc: 0.99 - ETA: 10:53 - loss: 0.1635 - acc: 0.99 - ETA: 10:51 - loss: 0.1635 - acc: 0.99 - ETA: 10:49 - loss: 0.1637 - acc: 0.99 - ETA: 10:47 - loss: 0.1637 - acc: 0.99

563/563 [==============================] - ETA: 4:57 - loss: 0.1589 - acc: 0.997 - ETA: 4:55 - loss: 0.1590 - acc: 0.997 - ETA: 4:53 - loss: 0.1590 - acc: 0.997 - ETA: 4:52 - loss: 0.1590 - acc: 0.997 - ETA: 4:50 - loss: 0.1590 - acc: 0.997 - ETA: 4:48 - loss: 0.1590 - acc: 0.997 - ETA: 4:46 - loss: 0.1589 - acc: 0.997 - ETA: 4:44 - loss: 0.1589 - acc: 0.997 - ETA: 4:42 - loss: 0.1589 - acc: 0.997 - ETA: 4:40 - loss: 0.1590 - acc: 0.997 - ETA: 4:38 - loss: 0.1590 - acc: 0.997 - ETA: 4:36 - loss: 0.1589 - acc: 0.997 - ETA: 4:34 - loss: 0.1589 - acc: 0.997 - ETA: 4:32 - loss: 0.1589 - acc: 0.997 - ETA: 4:30 - loss: 0.1589 - acc: 0.997 - ETA: 4:28 - loss: 0.1589 - acc: 0.997 - ETA: 4:26 - loss: 0.1589 - acc: 0.997 - ETA: 4:24 - loss: 0.1589 - acc: 0.997 - ETA: 4:23 - loss: 0.1589 - acc: 0.997 - ETA: 4:21 - loss: 0.1589 - acc: 0.997 - ETA: 4:19 - loss: 0.1589 - acc: 0.997 - ETA: 4:17 - loss: 0.1589 - acc: 0.997 - ETA: 4:15 - loss: 0.1589 - acc: 0.997 - ETA: 4:13 - loss: 0.1588 - acc: 0.997

204/563 [=========>....................] - ETA: 18:01 - loss: 0.1332 - acc: 1.00 - ETA: 17:52 - loss: 0.1338 - acc: 1.00 - ETA: 17:56 - loss: 0.1382 - acc: 1.00 - ETA: 17:54 - loss: 0.1375 - acc: 1.00 - ETA: 17:57 - loss: 0.1374 - acc: 1.00 - ETA: 17:55 - loss: 0.1372 - acc: 1.00 - ETA: 17:57 - loss: 0.1364 - acc: 1.00 - ETA: 17:58 - loss: 0.1364 - acc: 1.00 - ETA: 17:55 - loss: 0.1359 - acc: 1.00 - ETA: 17:55 - loss: 0.1352 - acc: 1.00 - ETA: 17:55 - loss: 0.1348 - acc: 1.00 - ETA: 17:52 - loss: 0.1346 - acc: 1.00 - ETA: 17:50 - loss: 0.1341 - acc: 1.00 - ETA: 17:48 - loss: 0.1339 - acc: 1.00 - ETA: 17:46 - loss: 0.1336 - acc: 1.00 - ETA: 17:45 - loss: 0.1336 - acc: 1.00 - ETA: 17:42 - loss: 0.1332 - acc: 1.00 - ETA: 17:40 - loss: 0.1329 - acc: 1.00 - ETA: 17:39 - loss: 0.1325 - acc: 1.00 - ETA: 17:36 - loss: 0.1320 - acc: 1.00 - ETA: 17:34 - loss: 0.1316 - acc: 1.00 - ETA: 17:32 - loss: 0.1314 - acc: 1.00 - ETA: 17:30 - loss: 0.1313 - acc: 1.00 - ETA: 17:28 - loss: 0.1312 - acc: 1.00

408/563 [====================>.........] - ETA: 11:33 - loss: 0.1250 - acc: 0.99 - ETA: 11:31 - loss: 0.1250 - acc: 0.99 - ETA: 11:29 - loss: 0.1251 - acc: 0.99 - ETA: 11:27 - loss: 0.1250 - acc: 0.99 - ETA: 11:25 - loss: 0.1251 - acc: 0.99 - ETA: 11:24 - loss: 0.1250 - acc: 0.99 - ETA: 11:22 - loss: 0.1251 - acc: 0.99 - ETA: 11:20 - loss: 0.1251 - acc: 0.99 - ETA: 11:18 - loss: 0.1251 - acc: 0.99 - ETA: 11:16 - loss: 0.1251 - acc: 0.99 - ETA: 11:14 - loss: 0.1252 - acc: 0.99 - ETA: 11:12 - loss: 0.1253 - acc: 0.99 - ETA: 11:10 - loss: 0.1255 - acc: 0.99 - ETA: 11:08 - loss: 0.1255 - acc: 0.99 - ETA: 11:06 - loss: 0.1257 - acc: 0.99 - ETA: 11:04 - loss: 0.1258 - acc: 0.99 - ETA: 11:02 - loss: 0.1258 - acc: 0.99 - ETA: 11:00 - loss: 0.1258 - acc: 0.99 - ETA: 10:58 - loss: 0.1259 - acc: 0.99 - ETA: 10:56 - loss: 0.1259 - acc: 0.99 - ETA: 10:54 - loss: 0.1260 - acc: 0.99 - ETA: 10:52 - loss: 0.1261 - acc: 0.99 - ETA: 10:50 - loss: 0.1261 - acc: 0.99 - ETA: 10:48 - loss: 0.1261 - acc: 0.99

563/563 [==============================] - ETA: 4:58 - loss: 0.1375 - acc: 0.997 - ETA: 4:56 - loss: 0.1375 - acc: 0.997 - ETA: 4:54 - loss: 0.1374 - acc: 0.997 - ETA: 4:52 - loss: 0.1374 - acc: 0.997 - ETA: 4:50 - loss: 0.1374 - acc: 0.997 - ETA: 4:48 - loss: 0.1374 - acc: 0.997 - ETA: 4:46 - loss: 0.1373 - acc: 0.997 - ETA: 4:44 - loss: 0.1373 - acc: 0.997 - ETA: 4:42 - loss: 0.1373 - acc: 0.997 - ETA: 4:40 - loss: 0.1372 - acc: 0.997 - ETA: 4:38 - loss: 0.1373 - acc: 0.997 - ETA: 4:36 - loss: 0.1372 - acc: 0.997 - ETA: 4:34 - loss: 0.1372 - acc: 0.997 - ETA: 4:32 - loss: 0.1371 - acc: 0.997 - ETA: 4:31 - loss: 0.1371 - acc: 0.997 - ETA: 4:29 - loss: 0.1371 - acc: 0.997 - ETA: 4:27 - loss: 0.1371 - acc: 0.997 - ETA: 4:25 - loss: 0.1371 - acc: 0.997 - ETA: 4:23 - loss: 0.1370 - acc: 0.997 - ETA: 4:21 - loss: 0.1370 - acc: 0.997 - ETA: 4:19 - loss: 0.1370 - acc: 0.997 - ETA: 4:17 - loss: 0.1370 - acc: 0.997 - ETA: 4:15 - loss: 0.1370 - acc: 0.997 - ETA: 4:13 - loss: 0.1370 - acc: 0.997